<a href="https://colab.research.google.com/github/pavi-ninjaac/NuralNetwork_scratch/blob/main/FullyConnectedLayer/FullyConnectedLayer_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#gonna train the XOR gate from scratch
import numpy as np

In [2]:
class FClayer:
  def __init__(self , input_size , output_size):
    self.input = None
    self.output = None
    #rondomly initialize the weights
    self.weights = np.random.rand(input_size , output_size) - 0.5
    self.bias =  np.random.rand(1,output_size) - 0.5 

  def forward_propogation(self , input):
    self.input = input
    self.output = np.dot(self.input , self.weights) + self.bias
    return self.output

  def back_propogation(self , output_error , learning_rate):
    input_error = np.dot(output_error , self.weights.T)
    weights_error = np.dot(self.input.T , output_error)
    bias_error = output_error
    #update the weights and bias
    self.weights -= learning_rate *  weights_error
    self.bias -= learning_rate * bias_error

    return input_error

In [4]:
#activation layer
#inherite the fully connected layer



class ActivationLayer(FClayer):
  def __init__(self, actvation , activation_derivated):
    self.actvation = actvation
    self.activation_derivated = activation_derivated

  def forward_propogation(self , input_data):
    self.input = input_data
    self.output = self.actvation(self.input)
    return self.output

  def back_propogation(self ,  output_error , learning_rate):
    return self.activation_derivated(self.input) . output_error

In [5]:
def tanh(x):
  return np.tanh(x)

def tanh_derivated(x):
  return 1-np.tanh(x) ** 2

In [6]:
#defining the lose

def mse(y_true , y_pred):
  return np.mean(np.power(y_true - y_pred , 2))

def mse_derivated(y_true , y_pred):
  return 2*(y_pred - y_true) / y_true.size

In [8]:
#defning the network or model

class model:
  def __init__(self):
    self.layers = []
    self.loss = None
    self.loss_derivated = None

  #adding layer to the model
  def add(self , layer):
    self.layers.append(layer)

  def setloss(self, loss , loss_derivated):
    self.loss = loss
    self.loss_derivated = loss_derivated

  #predict the unknow data
  def predict(self , Newinput):
    samples = len(Newinput)
    result = []
    for i in range(samples):
      output = Newinput[i]
      
      for layer in self.layers:
        ouput = layer.forward_propogation(output)

      result.append(output)
    return result
  
  def fit(self , x_train , y_train , epoches , learning_rate):
    sampels = len(x_train)
    for i in epoches:
      err = 0
      for j in range(sampels):
        output = x_train[j] 
        for layer in self.layers:
          ouput = layer.forward_propogation(output)
        
        #compute the loss
        err += self.loss(y_train[j] , output)

        #backpropogation
        error = self.loss_derivated(y_train[j] , output)

        #update the weights
        for layer in reversed(self.layers):
          error = layer.back_propogation(error , learning_rate)
        
    #calculate the average error and display that
      err /= sampels
      print('epoches..'+j+'/'+epoches+'-------'+'Error'+err)
